In [1]:
import json
import difflib
import re
from datetime import datetime
import os


from langchain.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

# 設定 Markdown 根目錄與向量儲存資料夾
markdown_folder = r"TSpec-LLM/Rel-16"
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device': 'cuda'}
embedding = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

# === 解析 conf 成 dict（簡易巢狀處理） ===
def parse_conf_to_dict(path):
    conf_dict = {}
    stack = [conf_dict]

    with open(path, 'r', encoding='utf-8', errors='ignore') as f:
        lines = f.readlines()

    for line in lines:
        line = line.strip()
        if not line or line.startswith('#'):
            continue
        if '=' in line and '{' not in line:
            key, val = line.split('=', 1)
            key = key.strip()
            val = val.strip().rstrip(';')
            stack[-1][key] = val
        elif '=' in line and '{' in line:
            key = line.split('=')[0].strip()
            new_dict = {}
            stack[-1][key] = new_dict
            stack.append(new_dict)
        elif '};' in line or '}' in line:
            if len(stack) > 1:
                stack.pop()
    return conf_dict

# === 找出 conf 差異 ===
def extract_conf_diff(correct_conf, error_conf):
    d1 = parse_conf_to_dict(correct_conf)
    d2 = parse_conf_to_dict(error_conf)
    diff_keys = []

    def compare_dicts(a, b, prefix=""):
        for key in a:
            if key not in b:
                diff_keys.append(prefix + key)
            elif isinstance(a[key], dict) and isinstance(b[key], dict):
                compare_dicts(a[key], b[key], prefix + key + ".")
            elif a[key] != b[key]:
                diff_keys.append(prefix + key)
        for key in b:
            if key not in a:
                diff_keys.append(prefix + key)

    compare_dicts(d1, d2)
    return diff_keys

# === 擷取錯誤 log snippet ===
def extract_log_diff(correct_log, error_log):
    with open(correct_log, 'r', encoding='utf-8', errors='ignore') as f1, \
         open(error_log, 'r', encoding='utf-8', errors='ignore') as f2:
        lines1 = f1.readlines()
        lines2 = f2.readlines()

    diff = list(difflib.unified_diff(lines1, lines2))
    added_lines = [line[1:].strip() for line in diff if line.startswith('+') and not line.startswith('+++')]
    err_lines = [l for l in added_lines if any(e in l.lower() for e in ['error', 'fail', 'assert', 'syntax', 'invalid',"yes","no"])]
    return err_lines[:3]

def cu_conf_to_json(conf_path, output_json_path):
    with open(conf_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    result = []
    assign_pattern = re.compile(r'^\s*([A-Za-z0-9_]+)\s*=\s*(.+?);\s*(#.*)?$')

    for line in lines:
        stripped = line.strip()
        if stripped.startswith("#") or not stripped:
            continue  # Skip comment or empty lines

        match = assign_pattern.match(stripped)
        if match:
            key = match.group(1)
            value = match.group(2).strip()
            full_content = f"{key} = {value};"
            result.append({
                "label": key,
                "content": full_content
            })

    with open(output_json_path, 'w', encoding='utf-8') as f:
        json.dump(result, f, indent=2, ensure_ascii=False)

    print(f"✅ JSON saved to: {output_json_path}")

def du_conf_to_json(conf_path, output_json_path):
    results = []

    with open(conf_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    for line in lines:
        # 移除註解與前後空白
        line = line.strip()
        if not line or line.startswith("#") or line.startswith("//"):
            continue

        # 正規化抓取 label 與 content（只抓最外層參數）
        match = re.match(r'^([a-zA-Z0-9_]+)\s*=\s*.+;', line)
        if match:
            label = match.group(1)
            results.append({
                "label": label,
                "content": line
            })

    # 儲存為 JSON
    with open(output_json_path, 'w', encoding='utf-8') as outfile:
        json.dump(results, outfile, indent=2)
    print(f"✅ JSON saved to {output_json_path}")

def query_all_subdbs(query_text, root_path, k=1):
    results = []
    for sub in os.listdir(root_path):
        sub_path = os.path.join(root_path, sub)
        if not os.path.isdir(sub_path): continue
        db = Chroma(persist_directory=sub_path, embedding_function=embedding)
        try:
            res = db.similarity_search_with_score(query_text, k=k)
            results.extend([(doc, score, sub) for doc, score in res])
        except:
            continue
    # 依照分數排序
    sorted_results = sorted(results, key=lambda x: x[1])[:k]
    return sorted_results


2025-08-15 01:50:10.904469: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-15 01:50:10.923146: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-15 01:50:10.923167: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-15 01:50:10.923766: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-15 01:50:10.927019: I tensorflow/core/platform/cpu_feature_guar

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

2025-08-15 01:50:11.331190: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

# NVDIA API (LLM seeting)

In [ ]:
from pathlib import Path
from openai import OpenAI
import google.generativeai as genai
from google.generativeai.types import GenerationConfig


# === 呼叫 NVIDIA LLM API ===
def call_nvidia_llm(prompt: str) -> str:
    client = OpenAI(
        base_url = "https://integrate.api.nvidia.com/v1",
        api_key = ""
    )
    response = client.chat.completions.create(
            model="microsoft/phi-4-multimodal-instruct",
            messages = [
            {"role": "user", "content": prompt}
        ],
            temperature=0.6,
            top_p=0.95,
            max_tokens=2048
        )

    return response.choices[0].message.content

def call_google_llm(prompt: str) -> str:
    # 建議用環境變數提供金鑰：export GEMINI_API_KEY="你的金鑰"
    genai.configure(api_key = "")

    model = genai.GenerativeModel(
        model_name="gemini-1.5-pro-002",
        system_instruction="detailed thinking on" # 對應原本的 system role
    )

    gen_config = GenerationConfig(
        temperature=0.6,
        top_p=0.95,
        max_output_tokens=4096,
        response_mime_type="text/plain"
    )

    # 最單純的一次性生成
    resp = model.generate_content(prompt, generation_config=gen_config)
    texts = []
    try:
        for cand in (resp.candidates or []):
            # cand.finish_reason: STOP / MAX_TOKENS / SAFETY / ...
            content = getattr(cand, "content", None)
            if not content:
                continue
            parts = getattr(content, "parts", None) or []
            for p in parts:
                t = getattr(p, "text", None)
                if t:
                    texts.append(t)
    except Exception as e:
        # 落地回報，方便除錯
        raise RuntimeError(f"Failed to parse response parts: {e}")

    if texts:
        return "\n".join(texts).strip()

    # 沒文字就給出可診斷資訊
    finish_reasons = [getattr(c, "finish_reason", None) for c in (resp.candidates or [])]
    safety = [getattr(c, "safety_ratings", None) for c in (resp.candidates or [])]
    prompt_fb = getattr(resp, "prompt_feedback", None)

    debug_msg = (
        "Empty response.text (no Parts with text).\n"
        f"finish_reasons={finish_reasons}\n"
        f"safety_ratings={safety}\n"
        f"prompt_feedback={prompt_fb}\n"
    )
    raise ValueError(debug_msg)




# Huggingface API 

In [3]:
from langchain.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

# 設定 embedding 模型（GPU）
embedding = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cuda'}
)

# 連接已儲存的 DB
db_path = "/home/aiml/johnson/thesis_rag/3gpp_db"
vectordb = Chroma(persist_directory=db_path, embedding_function=embedding)

/tmp/ipykernel_453399/164606561.py:12: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory=db_path, embedding_function=embedding)


In [4]:
baseline_dir = "/home/aiml/johnson/auto_gen_debug_yaml/success_data"
fail_dir = "/home/aiml/johnson/auto_gen_debug_yaml/fail_data"
none_rag_debug_yaml_dir = "/home/aiml/johnson/auto_gen_debug_yaml/debug_yaml_dir/none_3gpp_rag"
rag_debug_yaml_dir = "/home/aiml/johnson/auto_gen_debug_yaml/debug_yaml_dir/3gpp_rag"

test_cu_index = "2_1_cu_modified_gNB_ID"
# test_du_index = "22_1_du_modified_dl_absoluteFrequencyPointA"



# === 設定檔案路徑 ===
baseline_cu_conf = Path(f"{baseline_dir}/conf/0_cu_gnb_success.conf")
baseline_du_conf = Path(f"{baseline_dir}/conf/0_du_gnb_success.conf")
baseline_cu_log = Path(f"{baseline_dir}/log/0_cu_gnb_success.log")
baseline_du_log = Path(f"{baseline_dir}/log/0_du_gnb_success.log")

# === CU/DU 路徑 ===
error_cu_conf = Path(f"{fail_dir}/conf/cu_confs/{test_cu_index}.conf")
error_cu_log = Path(f"{fail_dir}/log/cu/{test_cu_index}_cu.log")

# error_du_conf = Path(f"{fail_dir}/conf/du_confs/{test_du_index}.conf")
# error_du_log = Path(f"{fail_dir}/log/du/{test_du_index}_du.log")


output_yaml = Path("debug_generated.yaml")
cases = []


# === CU 差異 ===
print("=== [CU Config Diff] ===")
cu_conf_diff = extract_conf_diff(baseline_cu_conf, error_cu_conf)
for k in cu_conf_diff:
    print(f"- {k}")

print("\n=== [CU Log Diff] ===")
cu_log_diff = extract_log_diff(baseline_cu_log, error_cu_log)
for line in cu_log_diff:
    print(f">>> {line}")

# # === DU 差異 ===
# print("\n=== [DU Config Diff] ===")
# du_conf_diff = extract_conf_diff(baseline_du_conf, error_du_conf)
# for k in du_conf_diff:
#     print(f"- {k}")

# print("\n=== [DU Log Diff] ===")
# du_log_diff = extract_log_diff(baseline_du_log, error_du_log)
# for line in du_log_diff:
#     print(f">>> {line}")

# print("--------------------------")


cu_conf_diff_text = "\n".join(f"- {k}" for k in cu_conf_diff)
cu_log_diff_text = "\n".join(f"- {l}" for l in cu_log_diff[:5]) 
# du_conf_diff_text = "\n".join(f"- {k}" for k in du_conf_diff)
# du_log_diff_text = "\n".join(f"- {l}" for l in du_log_diff[:5]) 


=== [CU Config Diff] ===
- gNB_ID

=== [CU Log Diff] ===
>>> Assertion (config_isparamset(gnbParms, 0)) failed!
>>> gNB_ID is not defined in configuration file
>>> ../../../openair2/E1AP/e1ap_setup.c:132 RCconfig_NR_CU_E1() Exiting OAI softmodem: _Assert_Exit_


In [5]:
summary_prompt = f"""### Role
You are a professional 5G network debugging assistant.

### Task Description
Your task is to analyze the differences between a working (baseline) and a failed CU/DU configuration, including error logs. From this information, generate a structured summary of the misconfiguration in a way that facilitates both human understanding and automated processing.

### Background Context
The baseline configuration and logs come from a functioning CU/DU setup. The failed version contains slight parameter modifications that caused system errors. These changes are shown below and may affect areas such as MIMO layer setup, SSB configuration, SCTP/IP bindings, etc.

You must identify which configuration parameter(s) introduced the error, what part of the system (CU or DU) is affected, and which error log messages clearly reflect the failure.

---

[CONFIG DIFFERENCE]
CU:
{cu_conf_diff_text or "No difference detected."}

[ERROR LOG]
CU:
{cu_log_diff_text or "No error log from CU."}
---

### Expected Output
Please output your response using the following structured format:

**Summary:**
<Concise explanation of what went wrong>

**Affected Component:** <CU / DU>

**Configuration Parameter(s):**
- <parameter1>
- <parameter2> (if applicable)

**Error Log Lines:**
- "<log line 1>"
- "<log line 2>"

**Error Cause:**
<Technical explanation of why the misconfiguration caused the error, referencing relevant 5G concepts or functions (e.g., RCconfig_nr_macrlc)>

**Semantic Query Version:**
Provide 2 alternate 1-2 sentence phrases that could be used as semantic queries to retrieve related 3GPP spec chunks or debug cases.
"""


In [6]:
print(summary_prompt)

### Role
You are a professional 5G network debugging assistant.

### Task Description
Your task is to analyze the differences between a working (baseline) and a failed CU/DU configuration, including error logs. From this information, generate a structured summary of the misconfiguration in a way that facilitates both human understanding and automated processing.

### Background Context
The baseline configuration and logs come from a functioning CU/DU setup. The failed version contains slight parameter modifications that caused system errors. These changes are shown below and may affect areas such as MIMO layer setup, SSB configuration, SCTP/IP bindings, etc.

You must identify which configuration parameter(s) introduced the error, what part of the system (CU or DU) is affected, and which error log messages clearly reflect the failure.

---

[CONFIG DIFFERENCE]
CU:
- gNB_ID

[ERROR LOG]
CU:
- Assertion (config_isparamset(gnbParms, 0)) failed!
- gNB_ID is not defined in configuration fil

In [7]:
llm_summary_output = call_nvidia_llm(summary_prompt)
print(llm_summary_output)

# llm_summary_output = call_google_llm(summary_prompt)
# print(llm_summary_output)

**Summary:**
The failed CU configuration lacks a defined `gNB_ID`, which is critical for the proper initialization and operation of the CU/DU connection.

**Affected Component:** CU

**Configuration Parameter(s):**
- gNB_ID

**Error Log Lines:**
- Assertion (config_isparamset(gnbParms, 0)) failed!
- gNB_ID is not defined in configuration file
- ../../../openair2/E1AP/e1ap_setup.c:132 RCconfig_NR_CU_E1() Exiting OAI softmodem: _Assert_Exit_

**Error Cause:**
The missing `gNB_ID` parameter in the CU configuration is essential for the UE (User Equipment) to identify the serving C-RAN (Cloud Radio Access Network) NodeB (gNB) and establish a connection. The error message indicates that the assertion check for the parameter set failed because `gNB_ID` was not present, leading to the premature exit of the OAI (OpenAirInterface) softmodem initialization process. This misconfiguration disrupts the MIMO layer setup, SSB (Service-Specific Block) configuration, and SCTP/IP bindings, which are all 

In [8]:
db_path = "/home/aiml/johnson/old_thesis_rag/3gpp_db_300"
results = query_all_subdbs(llm_summary_output, db_path, k=3)

for i, (doc, score, source) in enumerate(results):
    print(f"\n🔹 Match {i+1} from [{source}] (Score: {score:.4f})")

for i, (doc, score, source) in enumerate(results):
    print(doc.metadata)

for i, (doc, score, source) in enumerate(results):
    print(doc.page_content[:500])

context_snippets = [
    f"[From {doc.metadata['spec']}]:\n{doc.page_content.strip()[:500]}"
    for doc, score, _ in results
]
match_info = "\n".join(
    [f"🔹 Match {i+1} from [{source}] (Score: {score:.4f})"
     for i, (doc, score, source) in enumerate(results)]
)

retrieved_context = "\n\n---\n\n".join(context_snippets)

final_output = match_info + "\n\n" + retrieved_context

print("--------------------------------------------------------------------------")
print(retrieved_context)

# 儲存到指定資料夾
out_dir = Path("/home/aiml/johnson/auto_gen_debug_yaml/debug_yaml_dir/3gpp_rag/top-k")
out_dir.mkdir(parents=True, exist_ok=True)

out_file = out_dir / f"{test_cu_index}_top{k}.txt"
with open(out_file, "w", encoding="utf-8") as f:
    f.write(final_output)

print(f"✅ Saved retrieved_context to {out_file}")

Number of requested results 3 is greater than number of elements in index 2, updating n_results = 2



🔹 Match 1 from [38463-gf0] (Score: 0.6726)

🔹 Match 2 from [38463-gf0] (Score: 0.6933)

🔹 Match 3 from [38463-gf0] (Score: 0.7554)
{'path': 'TSpec-LLM/Rel-16/38_series/38463-gf0.md', 'source': 'TSpec-LLM/Rel-16/38_series/38463-gf0.md', 'spec': '38463-gf0'}
{'path': 'TSpec-LLM/Rel-16/38_series/38463-gf0.md', 'source': 'TSpec-LLM/Rel-16/38_series/38463-gf0.md', 'spec': '38463-gf0'}
{'path': 'TSpec-LLM/Rel-16/38_series/38463-gf0.md', 'source': 'TSpec-LLM/Rel-16/38_series/38463-gf0.md', 'spec': '38463-gf0'}
--

-- **************************************************************

IMPORTS

Criticality,

ProcedureCode

FROM E1AP-CommonDataTypes

Reset,

ResetAcknowledge,

ErrorIndication,

GNB-CU-UP-E1SetupRequest,

GNB-CU-UP-E1SetupResponse,

GNB-CU-UP-E1SetupFailure,

GNB-CU-CP-E1SetupRequest,
--

-- GNB-CU-CP E1 Setup Failure

--

-- **************************************************************

GNB-CU-CP-E1SetupFailure ::= SEQUENCE {

protocolIEs ProtocolIE-Container { {GNB-CU-CP-E1SetupF

In [9]:
none_rag_debug_yaml_prompt = f"""
You are an intelligent debugging agent within the 5G OAI/O-RAN architecture. You have access to knowledge about 3GPP specs, O-RAN standards, and OAI gNB initialization logic.

Your task is to analyze failure cases in gNB (CU/DU) configuration. You will be given:
- Configuration file differences between a working and a failed setup
- Corresponding error log snippets from both CU and DU

Please perform the following:
1. Determine what caused the failure
2. Identify which config parameters are related
3. Extract representative log messages that indicate the failure
4. Generate a debug.yaml entry in the following format

Use 5G NR and O-RAN terminology in your explanation. Specify the affected protocol layer (e.g., PHY, MAC, RRC), and explain the root cause. If possible, refer to specific 3GPP specs (e.g., TS 38.331, 38.401) or OAI module behavior (e.g., RCconfig failure, DU startup path).

Use the following input:

[CONFIG DIFFERENCE]
CU:
{cu_conf_diff_text or "No difference detected."}


[ERROR LOG]
CU:
{cu_log_diff_text or "No error log from CU."}

Please output in this format:


Please output the following structure:

- stage: <cu_init / du_init / f1 / NGAP / cell search / random access / syntax error>
  type: <CU / DU>
  symptom: "<short explanation of failure>"
  log_snippet:
    - "..."
    - "..."
  related_config:
    - "<parameter1>"
  notes: |
    <Provide a technical explanation here using 5G/O-RAN context>

    
    
Important:
- Do NOT include triple backticks (```yaml or ```) in your output.
- Output only the raw YAML content, starting directly with the `- stage:` line.
"""

In [10]:
rag_debug_yaml_prompt = f"""
You are an intelligent debugging agent within the 5G OAI/O-RAN architecture. You have access to knowledge about 3GPP specs, O-RAN standards, and OAI gNB initialization logic.

In addition to the log and configuration diff, you are also given a set of relevant technical references retrieved from a document corpus. Use these references to improve your technical analysis and accuracy.

---

[RETRIEVED CONTEXT]
{retrieved_context}

---

Your task is to analyze failure cases in gNB (CU/DU) configuration. You will be given:
- Configuration file differences between a working and a failed setup
- Corresponding error log snippets from both CU and DU

Please perform the following:
1. Determine what caused the failure
2. Identify which config parameters are related
3. Extract representative log messages that indicate the failure
4. Generate a debug.yaml entry in the following format

Use 5G NR and O-RAN terminology in your explanation. Specify the affected protocol layer (e.g., PHY, MAC, RRC), and explain the root cause. If possible, refer to specific 3GPP specs (e.g., TS 38.331, 38.401) or OAI module behavior (e.g., RCconfig_nr_macrlc, f1ap_eNB_start).

---

[CONFIG DIFFERENCE]
CU:
{cu_conf_diff_text or "No difference detected."}


[ERROR LOG]
CU:
{cu_log_diff_text or "No error log from CU."}


---

Please output the following structure:

- stage: <cu_init / du_init / f1 / NGAP / cell search / random access / syntax error>
  type: <CU / DU>
  symptom: "<short explanation of failure>"
  log_snippet:
    - "..."
    - "..."
  related_config:
    - "<parameter1>"
  notes: |
    <Provide a technical explanation here using 5G/O-RAN context>

    

Important:
- Do NOT include triple backticks (```yaml or ```) in your output.
- Output only the raw YAML content, starting directly with the `- stage:` line.
"""


In [11]:
# print(none_rag_debug_yaml_prompt)

print(rag_debug_yaml_prompt)


You are an intelligent debugging agent within the 5G OAI/O-RAN architecture. You have access to knowledge about 3GPP specs, O-RAN standards, and OAI gNB initialization logic.

In addition to the log and configuration diff, you are also given a set of relevant technical references retrieved from a document corpus. Use these references to improve your technical analysis and accuracy.

---

[RETRIEVED CONTEXT]
[From 38463-gf0]:
--

-- **************************************************************

IMPORTS

Criticality,

ProcedureCode

FROM E1AP-CommonDataTypes

Reset,

ResetAcknowledge,

ErrorIndication,

GNB-CU-UP-E1SetupRequest,

GNB-CU-UP-E1SetupResponse,

GNB-CU-UP-E1SetupFailure,

GNB-CU-CP-E1SetupRequest,

---

[From 38463-gf0]:
--

-- GNB-CU-CP E1 Setup Failure

--

-- **************************************************************

GNB-CU-CP-E1SetupFailure ::= SEQUENCE {

protocolIEs ProtocolIE-Container { {GNB-CU-CP-E1SetupFailureIEs} },

...

GNB-CU-CP-E1SetupFailureIEs E1AP-PRO

In [12]:
none_rag_llm_output = call_nvidia_llm(none_rag_debug_yaml_prompt)
print(none_rag_llm_output)

print("-----"*100)
rag_llm_output = call_nvidia_llm(rag_debug_yaml_prompt)
print(rag_llm_output)


# none_rag_llm_output = call_google_llm(none_rag_debug_yaml_prompt)
# print(none_rag_llm_output)

# print("-----"*100)
# rag_llm_output = call_google_llm(rag_debug_yaml_prompt)
# print(rag_llm_output)





# 儲存到 debug_yaml_dir，以 timestamp 命名
none_rag_output_path = Path(none_rag_debug_yaml_dir)
rag_output_path = Path(rag_debug_yaml_dir)

none_rag_output_path.mkdir(parents=True, exist_ok=True)  # 確保資料夾存在
rag_output_path.mkdir(parents=True, exist_ok=True)  # 確保資料夾存在

none_output_file = none_rag_output_path / f"none_rag_debug_{test_cu_index}.yaml"
rag_output_file = rag_output_path / f"rag_debug_{test_cu_index}.yaml"

with open(none_output_file, "w", encoding="utf-8") as f:
    f.write(none_rag_llm_output)

with open(rag_output_file, "w", encoding="utf-8") as f:
    f.write(rag_llm_output)

- stage: cu_init
  type: CU
  symptom: "gNB_ID not found in configuration"
  log_snippet:
    - "Assertion (config_isparamset(gnbParms, 0)) failed!"
    - "gNB_ID is not defined in configuration file"
    - "../../../openair2/E1AP/e1ap_setup.c:132 RCconfig_NR_CU_E1() Exiting OAI softmodem: _Assert_Exit_"
  related_config:
    - "gNB_ID"
  notes: |
    The failure occurred during the CU initialization phase when the gNB_ID parameter was missing from the configuration file. According to 3GPP TS 38.401, which details the O-RAN architecture and interfaces, the gNB_ID is a crucial identifier for the gNB (gNodeB). Without this identifier, the CU cannot properly initialize and establish a connection with the DU. The error logs indicate that the configuration validation failed because the gNB_ID parameter was not set, leading to the termination of the OAI softmodem initialization process as per the OAI module behavior defined in 3GPP TS 38.401.

- stage: cu_init
  type: CU
  symptom: "gNB_ID m